In [ ]:
# 패키지 설치
!pip install yfinance --quiet

In [ ]:
# 패키지 임포트
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

In [ ]:
# ETF 데이터 다운로드
data = yf.download('SPY TLT', start='2010-01-01')
data.drop(['Open', 'High', 'Low', 'Close', 'Volume'], inplace=True, axis=1)
data = data.droplevel(0, axis=1)
rets = data.pct_change().fillna(0)

In [ ]:
rets.cumsum().plot()

# 성과지표

# 1. Sharpe Ratio

$$ SR = \frac{μ_{p} - r_{f}}{σ_{p}} $$

   $\mu_{p}$ : 포트폴리오의 평균수익률
   
   $r_{f}$ : 무위험수익률 (0으로 가정)
   
   $σ_{p}$ : 포트폴리오의 변동성 (표준편차)

   평균 수익률 & 변동성 지표 시간 척도 통일해줄 것

In [ ]:
# 샤프비율 계산
sharpe_ratio = rets.mean() * np.sqrt(252) / rets.std()
spy_sharpe_ratio = sharpe_ratio[0]
tlt_sharpe_ratio = sharpe_ratio[1]

print(spy_sharpe_ratio)
print(tlt_sharpe_ratio)

In [ ]:
# 1년 롤링 샤프비율 계산
lookback = 252 # 1년 주식 거래 일수
rolling_sharpe_ratio = rets.rolling(lookback).mean() * np.sqrt(252) / rets.rolling(lookback).std()

spy_rolling_sharpe = rolling_sharpe_ratio['SPY']
tlt_rolling_sharpe = rolling_sharpe_ratio['TLT']
print(spy_rolling_sharpe)
print(tlt_rolling_sharpe)

In [ ]:
# 샤프비율 시각화
plt.figure(figsize=(15, 8))
spy_rolling_sharpe.plot(label='SPY={}'.format(np.round(spy_sharpe_ratio, 2)))
tlt_rolling_sharpe.plot(label='TLT={}'.format(np.round(tlt_sharpe_ratio, 2)))
plt.axhline(spy_sharpe_ratio, color='g', linestyle='dashed', linewidth=2)
plt.axhline(tlt_sharpe_ratio, color='r', linestyle='dashed', linewidth=2)
plt.legend()
plt.title('Historical Sharpe Ratio')
plt.xlabel('Date')
plt.ylabel('1-Year Rolling Sharpe Ratio')

# 2. VaR 대비 성과 비율

$$ VaR Ratio = -\frac{\mu_{p} - r_{f}}{N * VaR_{δ, p}} $$

주의할 점
1. VaR을 마이너스로 계산하는 경우가 있기 때문에 지표를 양수로 만들기 위해서는 마이너스를 붙여주어야 함 
2. 평균 수익률과 VaR의 기간값 통일.
    > 평균 수익률이 연단위로 되어있으면, VaR 값에 N 곱해서 연율화


In [ ]:
# 각 ETF의 수익률 데이터
spy_rets = rets['SPY']
tlt_rets = rets['TLT']

# VaR 계산
delta = 0.01
spy_VaR = spy_rets.quantile(delta)
tlt_VaR = tlt_rets.quantile(delta)
spy_VaR, tlt_VaR

In [ ]:
# VaR 대비 성과 비율 계산
spy_VaR_ratio = - spy_rets.mean() / spy_VaR # 둘 다 일단위로 되어있으니 N 곱할 필요 x
tlt_VaR_ratio = - tlt_rets.mean() / tlt_VaR

In [ ]:
print(spy_VaR_ratio)
print(tlt_VaR_ratio)

In [ ]:
# 1년 롤링 VaR 대비 성과 비율 계산
spy_rolling_VaR_ratio = spy_rets.rolling(lookback).mean() / - spy_rets.rolling(lookback).quantile(delta)
tlt_rolling_VaR_ratio = tlt_rets.rolling(lookback).mean() / - tlt_rets.rolling(lookback).quantile(delta)

In [ ]:
# VaR 대비 성과 비율 시각화
plt.figure(figsize=(15, 8))
spy_rolling_VaR_ratio.plot(label='SPY={}'.format(np.round(spy_VaR_ratio, 2)))
tlt_rolling_VaR_ratio.plot(label='TLT={}'.format(np.round(tlt_VaR_ratio, 2)))
plt.axhline(spy_VaR_ratio, color='g', linestyle='dashed', linewidth=2)
plt.axhline(tlt_VaR_ratio, color='r', linestyle='dashed', linewidth=2)
plt.legend()
plt.xlabel('Date')
plt.ylabel('1-Year Rolling VaR Ratio')
plt.title('Historical Reward-to-VaR Ratio')

# 3. CVaR 대비 성과 비율 (Conditional VaR)

$$ CVaR Ratio = -\frac{μ_{p} - r_{f}}{CVaR_{δ, p}} $$

In [ ]:
def calculate_CVaR(rets, delta=0.01):
    VaR = rets.quantile(delta)
    return rets[rets <= VaR].mean()

In [ ]:
# CVaR 계산
spy_CVaR = calculate_CVaR(spy_rets)
tlt_CVaR = calculate_CVaR(tlt_rets)

# CVaR 대비 성과 비율 계산
spy_CVaR_ratio = spy_rets.mean() / -spy_CVaR
tlt_CVaR_ratio = tlt_rets.mean() / -tlt_CVaR

print(spy_CVaR_ratio)
print(tlt_CVaR_ratio)

In [ ]:
# 1년 롤링 CVaR 대비 성과 비율 계산
spy_rolling_CVaR_ratio = spy_rets.rolling(lookback).mean() / -spy_rets.rolling(lookback).apply(calculate_CVaR)
tlt_rolling_CVaR_ratio = tlt_rets.rolling(lookback).mean() / -tlt_rets.rolling(lookback).apply(calculate_CVaR)

In [ ]:
# CVaR 대비 성과 비율 시각화
plt.figure(figsize=(15, 8))
spy_rolling_CVaR_ratio.plot(label='SPY={}'.format(np.round(spy_CVaR, 2)))
tlt_rolling_CVaR_ratio.plot(label='TLT={}'.format(np.round(tlt_CVaR, 2)))
plt.axhline(spy_CVaR_ratio, color='g', linestyle='dashed', linewidth=2)
plt.axhline(tlt_CVaR_ratio, color='r', linestyle='dashed', linewidth=2)
plt.legend()
plt.xlabel('Date')
plt.ylabel('1-Year Rolling CVaR Ratio')
plt.title('Historical Reward-to-CVaR Ratio')

# 4. 승률(Hit Ratio)과 손익비(Gain-to-Pain Ratio)

승률
$$ HR = \frac{Σ_{i=1}^{N}1_{\{r_{i}>0\}}}{N} $$

손익비: 손실과 이익의 비율

$$ GPR = -\frac{Σ_{i=1}^{N}max(r_{i}, 0)}{Σ_{i=1}^{N}min(r_{i}, 0)} $$
'이겼을 때 얼마나 크게 이겼고, 졌을 때 얼마나 크게 졌는가'

손실은 음수기 때문에 마이너스 붙여서 지표 양수로 만들어줄 것.


# 5. 확률적 우위

확률적 우위가 존재하기 위한 조건
$$ GPR > \frac{1 - HR}{HR} $$

ex. 
매매를 2번해서 1번 성공하고 1번 실패를 했는데, 성공했을 때 이익이 손해보다 크면 잘한거지 그치

근데 이제 매매 2번 성공하고 1번 실패했는데, 하필 그 실패가 도지인거임. 그러면 매매를 계속해도 되는걸까 아니겠지 그치

확률적 우위가 존재하는 전략을 써야 장기적으로 손해를 안 본다.

In [ ]:
# 승률 계산 함수
def calculate_hit_ratio(rets):
    return len(rets[rets > 0.0]) / len(rets[rets != 0.0])

In [ ]:
# 승률 계산
spy_hit_ratio = calculate_hit_ratio(spy_rets)
tlt_hit_ratio = calculate_hit_ratio(tlt_rets)

print(spy_hit_ratio)
print(tlt_hit_ratio)

In [ ]:
# 롤링 승률 계산
spy_rolling_hit_ratio = spy_rets.rolling(252).apply(calculate_hit_ratio)
tlt_rolling_hit_ratio = tlt_rets.rolling(252).apply(calculate_hit_ratio)

In [ ]:
# 승률 시각화
plt.figure(figsize=(15, 5))
spy_rolling_hit_ratio.plot(label='SPY={}'.format(np.round(spy_hit_ratio, 2)))
tlt_rolling_hit_ratio.plot(label='TLT={}'.format(np.round(tlt_hit_ratio, 2)))
plt.axhline(spy_hit_ratio, color='g', linestyle='dashed', linewidth=2)
plt.axhline(tlt_hit_ratio, color='r', linestyle='dashed', linewidth=2)
plt.legend()
plt.xlabel('Date')
plt.ylabel('1-Year Rolling Hit Ratio')
plt.title('Historical Hit Ratio')

In [ ]:
# 손익비 계산 함수
def calculate_gtp_ratio(rets):
    return rets[rets > 0.0].mean() / -rets[rets < 0.0].mean()

In [ ]:
# 손익비 계산
spy_gtp_ratio = calculate_gtp_ratio(spy_rets)
tlt_gtp_ratio = calculate_gtp_ratio(tlt_rets)

print(spy_gtp_ratio)
print(tlt_gtp_ratio)

In [ ]:
# 롤링 손익비 계산
spy_rolling_gtp_ratio = spy_rets.rolling(252).apply(calculate_gtp_ratio)
tlt_rolling_gtp_ratio = tlt_rets.rolling(252).apply(calculate_gtp_ratio)

In [ ]:
# 손익비 시각화
plt.figure(figsize=(15, 5))
spy_rolling_gtp_ratio.plot(label='SPY={}'.format(np.round(spy_gtp_ratio, 2)))
tlt_rolling_gtp_ratio.plot(label='TLT={}'.format(np.round(tlt_gtp_ratio, 2)))
plt.axhline(spy_gtp_ratio, color='g', linestyle='dashed', linewidth=2)
plt.axhline(tlt_gtp_ratio, color='r', linestyle='dashed', linewidth=2)
plt.legend()
plt.xlabel('Date')
plt.ylabel('1-Year Rolling Gain-to-Pain Ratio')
plt.title('Historical Gain-to-Pain Ratio')

In [ ]:
# SPY 확률적 우위 검증
spy_TE = spy_gtp_ratio > (1 - spy_hit_ratio) / spy_hit_ratio
print(spy_TE)

In [ ]:
# TLT 확률적 우위 검증
tlt_TE = tlt_gtp_ratio > (1 - tlt_hit_ratio) / tlt_hit_ratio
print(tlt_TE)

# 팩터간 상관계수 분석

In [ ]:
import seaborn as sns
import pickle
import itertools as it
sns.set()

In [ ]:
# 데이터 로드
with open('factor.pkl', 'rb') as f:
    df = pickle.load(f)

In [ ]:
# 팩터 수익률 데이터프레임
factor_df = df.iloc[:, 5:]

In [ ]:
factor_df

In [ ]:
# 팩터별 누적 수익률 시각화
factor_df.cumsum().plot(legend=True, figsize=(12, 6), label=factor_df.columns)

In [ ]:
# 상관계수 행렬
corr = factor_df.corr()

# 상관계수 히트맵
plt.figure(figsize=(12, 8))
sns.heatmap(np.round(corr, 2), annot=True)

In [ ]:
# 팩터별 데이터프레임
beta_df = factor_df.iloc[:, :4]
momentum_df = factor_df.iloc[:, 4:8]
value_df = factor_df.iloc[:, 8:12]
carry_df = factor_df.iloc[:, 12:]

# 팩터별 포트폴리오
beta_port = beta_df.mean(axis=1)
momentum_port = momentum_df.mean(axis=1)
value_port = value_df.mean(axis=1)
carry_port = carry_df.mean(axis=1)

# 팩터별 포트폴리오 백테스팅 결과 시각화
plt.figure(figsize=(12, 6))
beta_port.cumsum().plot(label='Beta')
momentum_port.cumsum().plot(label='Momentum')
value_port.cumsum().plot(label='Value')
carry_port.cumsum().plot(label='Carry')
plt.legend()
plt.show()

In [ ]:
# 팩터 포트폴리오 수익률 분포 분석
factor_port_df = pd.concat([beta_port, momentum_port, value_port, carry_port], axis=1, join='inner')
factor_port_df.columns = ['beta', 'momentum', 'value', 'carry']

# 팩터 포트폴리오 왜도
print('Skew')
print(factor_port_df.skew())

# 팩터 포트폴리오 첨도
print('\n')
print('Kurtosis')
print(factor_port_df.kurt())

In [ ]:
# 자산배분 vs. 팩터배분
factor_port = factor_df.mean(axis=1)

# 그래프
plt.figure(figsize=(12, 6))
factor_port.cumsum().plot(label='Factor Portfolio')
beta_port.cumsum().plot(label='Beta Portfolio')
plt.legend()
plt.show()

In [ ]:
# 팩터 포트폴리오간 상관계수 히트맵
sns.heatmap(factor_port_df.corr(), annot=True)

In [ ]:
# 롤링 상관계수
cols = factor_port_df.columns
col_pairs = list(it.combinations(cols, 2))

lookback = 12 * 5
res = pd.DataFrame()

for pair in col_pairs:
    corr_name = f"{pair[0]}_{pair[1]}"
    res[corr_name] = factor_port_df[pair[0]].rolling(lookback).corr(factor_port_df[pair[1]])

res.plot(figsize=(12, 8))
res.mean(axis=1).plot(lw=5)